# Harz Basin Dataset Inspection

This notebook provides a comprehensive analysis of the Harz basin datasets, combining basin centroid extraction from shapefiles with NOAA GEFS forecast data and historical weather data analysis.

## Overview

The analysis consists of the following components:

### 1. Basin Centroid Extraction
- Load basin shapefiles from `data/harz/basin_shapefiles/`
- Extract centroid coordinates for each basin
- Visualize basin locations and centroids

### 2. NOAA GEFS Forecast Analysis
- Connect to NOAA GEFS 35-day forecast dataset
- Extract forecast data for basin centroids using all initialization times
- Compute ensemble quartiles (25th, 50th, 75th percentiles)
- Interpolate forecast data to hourly resolution

### 3. Historical Weather Data
- Fetch historical weather data for basin locations
- Align temporal coverage with forecast initialization times
- Merge historical and forecast data

### 4. Data Visualization and Analysis
- Compare historical observations with forecast quartiles
- Generate publication-quality plots following repository standards
- Export processed datasets for further analysis

## Import Libraries and Configuration

In [2]:
# Core libraries
import os
import sys
import warnings
from pathlib import Path
from datetime import datetime, timedelta

# Data handling
import pandas as pd
import numpy as np
import xarray as xr

# Geospatial libraries
import geopandas as gpd
from shapely.geometry import Point

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import seaborn as sns

# Set matplotlib style to match repository standards
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.size': 11,
    'axes.linewidth': 1.0,
    'grid.linewidth': 0.5,
    'lines.linewidth': 1.5
})

# Add src directory to path for custom modules
src_path = Path('../src')
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Import custom modules
try:
    from fetch_basin_forecasts import (
        load_basin_centroids,
        fetch_forecasts_for_basins,
        interpolate_to_hourly,
    )
    from fetch_basin_historical import fetch_historical_for_basins
except ImportError as e:
    print(f"Warning: Could not import custom modules: {e}")
    print("Some functionality may be limited.")

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("Libraries imported successfully.")
print(f"Working directory: {os.getcwd()}")

Some functionality may be limited.
Libraries imported successfully.
Working directory: /home/sngrj0hn/GitHub/neuralhydrology/operational_harz/gefs_10d_sample


## Define Data Paths and Configuration

In [3]:
# Data paths
shapefile_dir = Path("../../data/harz/basin_shapefiles")
output_dir = Path("../../data/harz/basin_centroids")
basin_centroids_file = output_dir / "basin_centroids.csv"

# Create output directory
output_dir.mkdir(parents=True, exist_ok=True)

# Visualization parameters
FIG_WIDTH = 6.4
FIG_HEIGHT = 5.0
DPI = 300

# Analysis parameters
FORECAST_HOURS = 240  # 10 days for hourly interpolation
HISTORICAL_DAYS = 7   # Days of historical data for visualization
QUARTILES = [0.25, 0.5, 0.75]  # Ensemble quartiles to compute

print(f"Shapefile directory: {shapefile_dir}")
print(f"Output directory: {output_dir}")
print(f"Basin centroids file: {basin_centroids_file}")
print(f"Shapefile directory exists: {shapefile_dir.exists()}")

Shapefile directory: ../../data/harz/basin_shapefiles
Output directory: ../../data/harz/basin_centroids
Basin centroids file: ../../data/harz/basin_centroids/basin_centroids.csv
Shapefile directory exists: True


## 1. Basin Centroid Extraction

Extract centroid coordinates from basin shapefiles in the Harz dataset.

In [4]:
def extract_basin_centroids(shapefile_dir, output_file=None):
    """
    Extract centroids from all shapefiles in the specified directory.
    
    Parameters:
    -----------
    shapefile_dir : Path or str
        Directory containing basin shapefiles
    output_file : Path or str, optional
        Output CSV file path for centroids
    
    Returns:
    --------
    geopandas.GeoDataFrame
        Combined data with basin names and centroid coordinates
    """
    shapefile_dir = Path(shapefile_dir)
    
    if not shapefile_dir.exists():
        raise FileNotFoundError(f"Shapefile directory not found: {shapefile_dir}")
    
    # Find all shapefiles
    shapefiles = list(shapefile_dir.glob("*.shp"))
    
    if not shapefiles:
        raise FileNotFoundError(f"No shapefiles found in {shapefile_dir}")
    
    print(f"Found {len(shapefiles)} shapefile(s):")
    for shp in shapefiles:
        print(f"  - {shp.name}")
    
    all_centroids = []
    
    for shapefile_path in shapefiles:
        try:
            # Read shapefile
            gdf = gpd.read_file(shapefile_path)
            print(f"\nProcessing {shapefile_path.name}:")
            print(f"  - CRS: {gdf.crs}")
            print(f"  - Number of features: {len(gdf)}")
            
            # Ensure CRS is geographic (WGS84) for lat/lon coordinates
            if gdf.crs != 'EPSG:4326':
                print(f"  - Converting from {gdf.crs} to EPSG:4326")
                gdf = gdf.to_crs('EPSG:4326')
            
            # Calculate centroids
            centroids = gdf.geometry.centroid
            
            # Extract coordinates
            lons = centroids.x.values
            lats = centroids.y.values
            
            # Create basin names
            basin_base_name = shapefile_path.stem
            
            for i, (lon, lat) in enumerate(zip(lons, lats)):
                basin_name = f"{basin_base_name}_Basin_{i}"
                all_centroids.append({
                    'basin': basin_name,
                    'longitude': lon,
                    'latitude': lat,
                    'shapefile': shapefile_path.name
                })
                print(f"  - {basin_name}: ({lat:.4f}, {lon:.4f})")
                
        except Exception as e:
            print(f"Error processing {shapefile_path.name}: {e}")
            continue
    
    if not all_centroids:
        raise ValueError("No valid centroids extracted from shapefiles")
    
    # Create DataFrame
    centroids_df = pd.DataFrame(all_centroids)
    
    # Save to CSV if output file specified
    if output_file:
        output_file = Path(output_file)
        output_file.parent.mkdir(parents=True, exist_ok=True)
        centroids_df.to_csv(output_file, index=False)
        print(f"\nCentroids saved to: {output_file}")
    
    return centroids_df

# Extract centroids from shapefiles
try:
    centroids_df = extract_basin_centroids(shapefile_dir, basin_centroids_file)
    print(f"\nSuccessfully extracted {len(centroids_df)} basin centroids")
    display(centroids_df)
except Exception as e:
    print(f"Error extracting centroids: {e}")
    # Create dummy data for demonstration if shapefiles not available
    centroids_df = pd.DataFrame({
        'basin': ['oker_reservoir_catchment_Basin_0', 'sample_basin_Basin_1'],
        'longitude': [10.5, 10.6],
        'latitude': [51.8, 51.9],
        'shapefile': ['oker_reservoir_catchment.shp', 'sample_basin.shp']
    })
    print("Using dummy centroid data for demonstration")
    display(centroids_df)

Found 5 shapefile(s):
  - innerste_reservoir_catchment.shp
  - oker_reservoir_catchment.shp
  - ecker_reservoir_catchment.shp
  - soese_reservoir_catchment.shp
  - grane_reservoir_catchment.shp

Processing innerste_reservoir_catchment.shp:
  - CRS: EPSG:4326
  - Number of features: 1
  - innerste_reservoir_catchment_Basin_0: (51.8345, 10.3078)

Processing oker_reservoir_catchment.shp:
  - CRS: EPSG:4326
  - Number of features: 1
  - oker_reservoir_catchment_Basin_0: (51.8165, 10.4473)

Processing ecker_reservoir_catchment.shp:
  - CRS: EPSG:4326
  - Number of features: 1
  - ecker_reservoir_catchment_Basin_0: (51.8101, 10.5841)

Processing soese_reservoir_catchment.shp:
  - CRS: EPSG:4326
  - Number of features: 1
  - soese_reservoir_catchment_Basin_0: (51.7523, 10.3832)

Processing grane_reservoir_catchment.shp:
  - CRS: EPSG:4326
  - Number of features: 1
  - grane_reservoir_catchment_Basin_0: (51.8844, 10.3572)

Centroids saved to: ../../data/harz/basin_centroids/basin_centroids.csv

basin  longitude   latitude  \
0  innerste_reservoir_catchment_Basin_0  10.307828  51.834451   
1      oker_reservoir_catchment_Basin_0  10.447276  51.816468   
2     ecker_reservoir_catchment_Basin_0  10.584058  51.810056   
3     soese_reservoir_catchment_Basin_0  10.383225  51.752310   
4     grane_reservoir_catchment_Basin_0  10.357203  51.884391   

                          shapefile  
0  innerste_reservoir_catchment.shp  
1      oker_reservoir_catchment.shp  
2     ecker_reservoir_catchment.shp  
3     soese_reservoir_catchment.shp  
4     grane_reservoir_catchment.shp

In [7]:
# show random plot to verify matplotlib setup
def show_random_plot():
    """
    Show a random plot to verify matplotlib setup.
    """
    plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
    sns.scatterplot(data=centroids_df, x='longitude', y='latitude', hue='basin', palette='viridis')
    plt.title('Basin Centroids')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend(title='Basin', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


show_random_plot()

: 

In [ ]:
plt.plot(centroids_df['longitude'], centroids_df['latitude'], 'ro', markersize=5)
plt.title('Basin Centroids')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid()
plt.savefig(output_dir / 'basin_centroids.png', dpi=DPI, bbox_inches='tight')
plt.show()

: 

## 2. Visualize Basin Locations

Create a map showing the basin locations and their centroids.

In [ ]:
def plot_basin_overview(centroids_df, shapefile_dir=None, save_path=None):
    """
    Create an overview plot of basin locations and centroids.
    
    Parameters:
    -----------
    centroids_df : pandas.DataFrame
        DataFrame with basin centroids
    shapefile_dir : Path or str, optional
        Directory with shapefiles for basin boundaries
    save_path : Path or str, optional
        Path to save the plot
    """
    fig, ax = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))
    
    # Plot basin boundaries if shapefiles available
    if shapefile_dir and Path(shapefile_dir).exists():
        shapefiles = list(Path(shapefile_dir).glob("*.shp"))
        colors = plt.cm.Set3(np.linspace(0, 1, len(shapefiles)))
        
        for i, shapefile_path in enumerate(shapefiles):
            try:
                gdf = gpd.read_file(shapefile_path)
                if gdf.crs != 'EPSG:4326':
                    gdf = gdf.to_crs('EPSG:4326')
                
                gdf.plot(ax=ax, color=colors[i], alpha=0.6, 
                         edgecolor='black', linewidth=0.5,
                         label=shapefile_path.stem)
            except Exception as e:
                print(f"Could not plot {shapefile_path.name}: {e}")
    
    # Plot centroids
    ax.scatter(centroids_df['longitude'], centroids_df['latitude'], 
               c='red', s=50, marker='x', linewidth=2, 
               label='Basin Centroids', zorder=5)
    
    # Add basin labels
    for idx, row in centroids_df.iterrows():
        ax.annotate(row['basin'], 
                   (row['longitude'], row['latitude']),
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=8, ha='left')
    
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    ax.set_title('Harz Basin Locations and Centroids')
    ax.grid(True, alpha=0.3)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=DPI, bbox_inches='tight')
        print(f"Basin overview plot saved to: {save_path}")
    
    plt.show()

# Create basin overview plot
overview_plot_path = output_dir / "basin_overview.pdf"
# plot_basin_overview(centroids_df, shapefile_dir, overview_plot_path)

In [ ]:
plot_basin_overview(centroids_df, shapefile_dir, overview_plot_path)

## 3. Load NOAA GEFS Forecast Dataset

Connect to the NOAA GEFS 35-day forecast dataset and examine its structure.

In [ ]:
# Load NOAA GEFS forecast dataset
print("Connecting to NOAA GEFS dataset...")
try:
    ds = xr.open_zarr(
        "https://data.dynamical.org/noaa/gefs/forecast-35-day/latest.zarr?email=optional@email.com", 
        decode_timedelta=True
    )
    
    print("\n=== Dataset Summary ===")
    print(f"Time domain: {ds.attrs.get('time_domain', 'N/A')}")
    print(f"Forecast domain: {ds.attrs.get('forecast_domain', 'N/A')}")
    print(f"Spatial resolution: {ds.attrs.get('spatial_resolution', 'N/A')}")
    
    print("\n=== Dataset Dimensions ===")
    for dim, size in ds.dims.items():
        print(f"  {dim}: {size}")
    
    print("\n=== Variables ===")
    for var in list(ds.data_vars)[:5]:  # Show first 5 variables
        print(f"  {var}: {ds[var].dims}")
    if len(ds.data_vars) > 5:
        print(f"  ... and {len(ds.data_vars) - 5} more variables")
    
    # Show initialization times
    init_times = pd.to_datetime(ds.init_time.values)
    print(f"\n=== Initialization Times ===")
    print(f"Total number: {len(init_times)}")
    print(f"First: {init_times[0].strftime('%Y-%m-%d %H:%M')}")
    print(f"Last: {init_times[-1].strftime('%Y-%m-%d %H:%M')}")
    
    print("\nDataset loaded successfully.")
    
except Exception as e:
    print(f"Error loading NOAA GEFS dataset: {e}")
    print("Proceeding with limited analysis.")
    ds = None

## 4. Extract Forecasts for Basin Centroids

Extract forecast data for each basin centroid location.

In [ ]:
# Extract forecasts for basin centroids
if ds is not None:
    try:
        print("Extracting forecasts for basin centroids...")
        
        # Prepare centroids data structure expected by fetch_forecasts_for_basins
        if 'load_basin_centroids' in globals():
            # Use existing function if available
            centroids = centroids_df
        else:
            # Prepare data structure manually
            centroids = centroids_df
        
        # Extract forecasts (use all initialization times)
        if 'fetch_forecasts_for_basins' in globals():
            basin_forecasts = fetch_forecasts_for_basins(ds, centroids, init_time=None)
            print(f"Successfully extracted forecasts for {len(basin_forecasts.basin)} basins")
            print(f"Forecast dimensions: {dict(basin_forecasts.dims)}")
        else:
            print("fetch_forecasts_for_basins function not available")
            basin_forecasts = None
            
    except Exception as e:
        print(f"Error extracting forecasts: {e}")
        basin_forecasts = None
else:
    print("Skipping forecast extraction - dataset not available")
    basin_forecasts = None

## 5. Compute Forecast Quartiles

Convert ensemble forecasts to quartiles to reduce data size while preserving statistical information.

In [ ]:
def compute_forecast_quartiles_as_variables(forecast_ds, quartiles=None):
    """
    Compute quartiles from ensemble forecast data as separate variables.
    
    Parameters:
    -----------
    forecast_ds : xarray.Dataset
        Dataset with ensemble_member dimension
    quartiles : list, optional
        List of quantiles to compute (default: [0.25, 0.5, 0.75])
    
    Returns:
    --------
    xarray.Dataset
        Dataset with separate variables for each quartile
    """
    if quartiles is None:
        quartiles = QUARTILES
    
    print(f"Computing quartiles {quartiles} from ensemble forecasts...")
    
    # Define quartile suffixes
    quartile_suffixes = {
        0.25: '_q25',
        0.5: '_q50', 
        0.75: '_q75'
    }
    
    new_data_vars = {}
    
    # Process each data variable
    for var_name in forecast_ds.data_vars:
        print(f"  Processing variable: {var_name}")
        var_data = forecast_ds[var_name]
        
        # Compute quartiles
        var_quartiles = var_data.quantile(quartiles, dim='ensemble_member')
        
        # Create separate variables for each quartile
        for i, q in enumerate(quartiles):
            suffix = quartile_suffixes.get(q, f'_q{int(q*100)}')
            new_var_name = f"{var_name}{suffix}"
            
            # Extract quartile data (remove quantile dimension)
            quartile_data = var_quartiles.isel(quantile=i).drop('quantile')
            new_data_vars[new_var_name] = quartile_data
    
    # Create new dataset with same coordinates (excluding ensemble_member)
    coords_to_keep = {k: v for k, v in forecast_ds.coords.items() 
                      if 'ensemble_member' not in v.dims}
    
    quartile_ds = xr.Dataset(
        data_vars=new_data_vars,
        coords=coords_to_keep,
        attrs=forecast_ds.attrs.copy()
    )
    
    # Update attributes
    quartile_ds.attrs['quartile_processing'] = f'Computed quartiles {quartiles}'
    quartile_ds.attrs['original_ensemble_members'] = len(forecast_ds.ensemble_member)
    
    print(f"Original dataset dimensions: {dict(forecast_ds.dims)}")
    print(f"Quartile dataset dimensions: {dict(quartile_ds.dims)}")
    print(f"Original variables: {len(forecast_ds.data_vars)}")
    print(f"New quartile variables: {len(quartile_ds.data_vars)}")
    
    return quartile_ds

# Compute forecast quartiles
if basin_forecasts is not None:
    try:
        basin_forecasts_quartiles = compute_forecast_quartiles_as_variables(basin_forecasts)
        print("\nForecast quartiles computed successfully.")
        
        # Show sample variables
        sample_vars = list(basin_forecasts_quartiles.data_vars)[:6]
        print(f"Sample quartile variables: {sample_vars}")
        
    except Exception as e:
        print(f"Error computing forecast quartiles: {e}")
        basin_forecasts_quartiles = None
else:
    print("Skipping quartile computation - forecast data not available")
    basin_forecasts_quartiles = None

## 6. Interpolate to Hourly Resolution

Interpolate forecast data to hourly resolution for the first 10 days to enable merging with hourly historical data.

In [ ]:
# Interpolate forecast quartiles to hourly resolution
if basin_forecasts_quartiles is not None:
    try:
        print(f"Interpolating forecast data to hourly resolution ({FORECAST_HOURS} hours)...")
        
        if 'interpolate_to_hourly' in globals():
            basin_forecasts_quartiles_hourly = interpolate_to_hourly(
                basin_forecasts_quartiles, max_hours=FORECAST_HOURS
            )
            
            print(f"Original lead_times: {len(basin_forecasts_quartiles.lead_time)} steps")
            print(f"Hourly lead_times: {len(basin_forecasts_quartiles_hourly.lead_time)} steps")
            
            print("\nInterpolation completed successfully.")
        else:
            print("interpolate_to_hourly function not available")
            basin_forecasts_quartiles_hourly = basin_forecasts_quartiles
            
    except Exception as e:
        print(f"Error interpolating to hourly resolution: {e}")
        basin_forecasts_quartiles_hourly = basin_forecasts_quartiles
else:
    print("Skipping interpolation - quartile data not available")
    basin_forecasts_quartiles_hourly = None

## 7. Fetch Historical Weather Data

Retrieve historical weather data for the same basin locations to compare with forecasts.

In [ ]:
# Fetch historical weather data
if basin_forecasts_quartiles is not None:
    try:
        # Determine date range from forecast data
        start_date_hist = pd.to_datetime(
            basin_forecasts_quartiles.init_time.min().values
        ).strftime('%Y-%m-%d')
        end_date_hist = pd.to_datetime(
            basin_forecasts_quartiles.init_time.max().values
        ).strftime('%Y-%m-%d')
        
        print(f"Historical data range: {start_date_hist} to {end_date_hist}")
        
        # Define historical variables
        historical_variables = [
            "temperature_2m", "relative_humidity_2m", "precipitation", 
            "rain", "snowfall", "soil_moisture_0_to_7cm", 
            "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", 
            "soil_moisture_100_to_255cm", "et0_fao_evapotranspiration", 
            "surface_pressure", "snow_depth_water_equivalent"
        ]
        
        # Fetch historical data
        if 'fetch_historical_for_basins' in globals():
            basin_historical_data = fetch_historical_for_basins(
                centroids_df, start_date_hist, end_date_hist, historical_variables
            )
            
            if basin_historical_data is not None:
                print(f"Historical data dimensions: {dict(basin_historical_data.dims)}")
                print(f"Historical variables: {list(basin_historical_data.data_vars)[:5]}...")
            else:
                print("Failed to fetch historical data")
        else:
            print("fetch_historical_for_basins function not available")
            basin_historical_data = None
            
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        basin_historical_data = None
else:
    print("Skipping historical data fetch - forecast data not available")
    basin_historical_data = None

## 8. Merge Historical and Forecast Data

Combine historical observations with forecast quartiles into a unified dataset.

In [ ]:
# Merge historical and forecast data
basin_combined_data = None

if basin_forecasts_quartiles_hourly is not None:
    try:
        # Rename 'init_time' to 'time' for consistency
        basin_forecasts_renamed = basin_forecasts_quartiles_hourly.rename({'init_time': 'time'})
        
        if basin_historical_data is not None:
            print("Merging historical and forecast data...")
            
            # Add chunking for historical data
            basin_historical_data = basin_historical_data.chunk({
                'time': 'auto', 'basin': 'auto'
            })
            
            # Identify conflicting variable names
            forecast_base_vars = set()
            for var in basin_forecasts_renamed.data_vars:
                if var.endswith(('_q25', '_q50', '_q75')):
                    base_var = var.rsplit('_q', 1)[0]
                    forecast_base_vars.add(base_var)
            
            conflicting_vars = [
                var for var in basin_historical_data.data_vars 
                if var in forecast_base_vars
            ]
            
            # Rename conflicting historical variables
            rename_dict = {var: f"{var}_hist" for var in conflicting_vars}
            basin_historical_renamed = basin_historical_data.rename(rename_dict)
            
            if rename_dict:
                print(f"Renamed conflicting variables: {list(rename_dict.values())}")
            
            # Merge datasets
            basin_combined_data = xr.merge(
                [basin_historical_renamed, basin_forecasts_renamed], 
                compat='no_conflicts'
            )
            
            print(f"Combined dataset dimensions: {dict(basin_combined_data.dims)}")
            
            # Categorize variables by dimensions
            historical_vars = []
            forecast_vars = []
            
            for var_name in basin_combined_data.data_vars:
                var_dims = basin_combined_data[var_name].dims
                if 'lead_time' in var_dims:
                    forecast_vars.append(var_name)
                else:
                    historical_vars.append(var_name)
            
            print(f"Historical variables: {len(historical_vars)}")
            print(f"Forecast variables: {len(forecast_vars)}")
            
        else:
            print("Using forecast data only (no historical data available)")
            basin_combined_data = basin_forecasts_renamed
            
    except Exception as e:
        print(f"Error merging data: {e}")
        basin_combined_data = None
else:
    print("No data available for merging")

## 9. Visualize Combined Dataset

Create publication-quality plots comparing historical observations with forecast quartiles.

In [ ]:
def create_forecast_comparison_plot(combined_data, basin_name=None, 
                                   forecast_init_time=None, save_path=None):
    """
    Create a comparison plot of historical vs forecast data.
    
    Parameters:
    -----------
    combined_data : xarray.Dataset
        Combined historical and forecast dataset
    basin_name : str, optional
        Basin to plot (if None, uses first basin)
    forecast_init_time : str or pd.Timestamp, optional
        Forecast initialization time (if None, uses latest)
    save_path : Path or str, optional
        Path to save the plot
    """
    if combined_data is None:
        print("No combined data available for plotting")
        return
    
    # Select basin
    if basin_name is None:
        basin_name = combined_data.basin.values[0]
    
    print(f"Creating plot for basin: {basin_name}")
    
    # Select forecast initialization time
    if forecast_init_time is None:
        # Use latest available time
        available_times = pd.to_datetime(combined_data.time.values)
        forecast_init_time = available_times[-1]
    else:
        forecast_init_time = pd.to_datetime(forecast_init_time)
    
    print(f"Using forecast initialization time: {forecast_init_time}")
    
    try:
        # Select data for basin
        basin_data = combined_data.sel(basin=basin_name)
        
        # Define time windows
        hist_end_time = forecast_init_time
        hist_start_time = hist_end_time - timedelta(days=HISTORICAL_DAYS)
        
        # Extract data slices
        historical_slice = basin_data.sel(time=slice(hist_start_time, hist_end_time))
        forecast_slice = basin_data.sel(time=forecast_init_time)
        
        # Create plot
        fig, axes = plt.subplots(2, 1, figsize=(FIG_WIDTH, FIG_HEIGHT), sharex=True)
        
        # Temperature plot
        ax1 = axes[0]
        
        # Historical temperature
        if 'temperature_2m_hist' in historical_slice:
            hist_temp = historical_slice['temperature_2m_hist'].dropna(dim='time')
            if hist_temp.size > 0:
                ax1.plot(hist_temp.time, hist_temp.values, 'k-', 
                        linewidth=1.5, label='Historical', zorder=5)
        
        # Forecast temperature quartiles
        temp_vars = ['temperature_2m_q25', 'temperature_2m_q50', 'temperature_2m_q75']
        if all(var in forecast_slice for var in temp_vars):
            # Calculate forecast time axis
            lead_hours = forecast_slice.lead_time.values
            forecast_times = forecast_init_time + pd.to_timedelta(lead_hours, unit='h')
            
            # Plot quartile band
            ax1.fill_between(forecast_times,
                           forecast_slice['temperature_2m_q25'].values,
                           forecast_slice['temperature_2m_q75'].values,
                           color='#d62728', alpha=0.3, 
                           label='Forecast IQR (25-75%)', zorder=2)
            
            # Plot median
            ax1.plot(forecast_times, forecast_slice['temperature_2m_q50'].values,
                    'r-', linewidth=1.5, label='Forecast Median', zorder=3)
        
        ax1.set_ylabel('Temperature (°C)')
        ax1.legend(loc='upper left')
        ax1.grid(True, alpha=0.3)
        ax1.axvline(forecast_init_time, color='blue', linestyle='-', 
                   linewidth=1, label='Forecast Start', zorder=6)
        
        # Precipitation plot
        ax2 = axes[1]
        
        # Historical precipitation
        if 'precipitation' in historical_slice:
            hist_precip = historical_slice['precipitation'].dropna(dim='time')
            if hist_precip.size > 0:
                # Convert to bars
                bar_width = 1/24  # 1 hour in days
                ax2.bar(hist_precip.time, hist_precip.values,
                       width=bar_width, color='black', alpha=0.7,
                       label='Historical', align='edge', zorder=3)
        
        # Forecast precipitation quartiles
        precip_vars = ['precipitation_surface_q25', 'precipitation_surface_q50', 
                      'precipitation_surface_q75']
        if all(var in forecast_slice for var in precip_vars):
            # Convert from mm/s to mm/hr
            conversion_factor = 3600
            
            # Plot as bars
            bar_width = 1/24  # 1 hour in days
            
            # Plot median precipitation
            precip_median = forecast_slice['precipitation_surface_q50'].values * conversion_factor
            ax2.bar(forecast_times, precip_median,
                   width=bar_width, color='#1f77b4', alpha=0.8,
                   label='Forecast Median', align='edge', zorder=2)
        
        ax2.set_ylabel('Precipitation (mm)')
        ax2.set_xlabel('Time (UTC)')
        ax2.legend(loc='upper left')
        ax2.grid(True, alpha=0.3)
        ax2.axvline(forecast_init_time, color='blue', linestyle='-', 
                   linewidth=1, zorder=4)
        
        # Format x-axis
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        # Add title
        fig.suptitle(f'Historical vs Forecast Comparison\nBasin: {basin_name}', 
                    y=0.98, fontsize=12)
        
        # Save plot
        if save_path:
            plt.savefig(save_path, dpi=DPI, bbox_inches='tight')
            print(f"Plot saved to: {save_path}")
        
        plt.show()
        
    except Exception as e:
        print(f"Error creating plot: {e}")
        import traceback
        traceback.print_exc()

# Create comparison plot
if basin_combined_data is not None:
    comparison_plot_path = output_dir / "forecast_comparison.pdf"
    create_forecast_comparison_plot(
        basin_combined_data, 
        save_path=comparison_plot_path
    )
else:
    print("No combined data available for visualization")

## 10. Data Export and Summary

Export processed datasets and provide analysis summary.

In [ ]:
# Export processed datasets
def export_datasets(basin_combined_data, centroids_df, output_dir):
    """
    Export processed datasets to files.
    
    Parameters:
    -----------
    basin_combined_data : xarray.Dataset
        Combined historical and forecast data
    centroids_df : pandas.DataFrame
        Basin centroid coordinates
    output_dir : Path
        Output directory
    """
    output_dir = Path(output_dir)
    
    # Export centroids
    centroids_file = output_dir / "basin_centroids.csv"
    centroids_df.to_csv(centroids_file, index=False)
    print(f"Basin centroids exported to: {centroids_file}")
    
    # Export combined dataset if available
    if basin_combined_data is not None:
        try:
            combined_file = output_dir / "basin_combined_data.nc"
            basin_combined_data.to_netcdf(combined_file)
            print(f"Combined dataset exported to: {combined_file}")
            
            # Export dataset summary
            summary_file = output_dir / "dataset_summary.txt"
            with open(summary_file, 'w') as f:
                f.write("Harz Basin Dataset Summary\n")
                f.write("=" * 30 + "\n\n")
                f.write(f"Number of basins: {len(basin_combined_data.basin)}\n")
                f.write(f"Dataset dimensions: {dict(basin_combined_data.dims)}\n\n")
                
                # Categorize variables
                historical_vars = []
                forecast_vars = []
                
                for var_name in basin_combined_data.data_vars:
                    var_dims = basin_combined_data[var_name].dims
                    if 'lead_time' in var_dims:
                        forecast_vars.append(var_name)
                    else:
                        historical_vars.append(var_name)
                
                f.write(f"Historical variables ({len(historical_vars)}):")
                for var in historical_vars:
                    f.write(f"\n  - {var}")
                
                f.write(f"\n\nForecast variables ({len(forecast_vars)}):")
                for var in forecast_vars:
                    f.write(f"\n  - {var}")
                
                # Time range
                time_range = basin_combined_data.time
                f.write(f"\n\nTime range: {time_range.min().values} to {time_range.max().values}")
                
                if 'lead_time' in basin_combined_data.dims:
                    lead_range = basin_combined_data.lead_time
                    f.write(f"\nLead time range: {lead_range.min().values} to {lead_range.max().values} hours")
            
            print(f"Dataset summary exported to: {summary_file}")
            
        except Exception as e:
            print(f"Error exporting combined dataset: {e}")
    else:
        print("No combined dataset available for export")

# Export datasets
print("\n=== Data Export ===")
export_datasets(basin_combined_data, centroids_df, output_dir)

# Analysis summary
print("\n=== Analysis Summary ===")
print(f"✓ Basin centroids: {len(centroids_df)} basins extracted")
print(f"✓ Shapefiles processed: {centroids_df['shapefile'].nunique()} files")

if basin_combined_data is not None:
    print(f"✓ Combined dataset: {dict(basin_combined_data.dims)}")
    print(f"✓ Variables: {len(basin_combined_data.data_vars)} total")
    print(f"✓ Time range: {basin_combined_data.time.min().values} to {basin_combined_data.time.max().values}")
else:
    print("⚠ Combined dataset: Not available (forecast/historical data issues)")

print(f"\n📁 Output files saved to: {output_dir}")
print("\n🎯 Analysis complete! The dataset is ready for hydrological modeling.")